In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style as style # for styling the graphs
#_______________________________________________________________________________________________________________
# style.available (to know the available list of styles)
style.use('ggplot') # chosen style
plt.rc('xtick',labelsize=13) # to globally set the tick size
plt.rc('ytick',labelsize=13) # to globally set the tick size
# To print multiple outputs together
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# Change column display number during print
pd.set_option('display.max_columns', 500)
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')
# To display float with 2 decimal, avoid scientific printing
pd.options.display.float_format = '{:.2f}'.format
import seaborn as sns
import warnings
import sweetviz
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_recall_curve, auc, roc_auc_score, roc_curve, recall_score, classification_report
from sklearn.metrics import roc_auc_score ,mean_squared_error,accuracy_score,classification_report,roc_curve,confusion_matrix,precision_score,f1_score
import itertools
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier


In [2]:
path='./Train.csv'
path1='./Test.csv'

In [3]:
train=pd.read_csv(path)
train

,Team1,Team2,Stadium,HostCountry,Team1_Venue,Team2_Venue,Team1_Innings,Team2_Innings,MonthOfMatch,MatchWinner
0,5,4,37,4,Home,Away,Second,First,Dec,4
1,1,14,84,7,Neutral,Neutral,First,Second,Sep,1
2,9,15,47,9,Home,Away,First,Second,Feb,9
3,7,2,102,6,Home,Away,First,Second,Aug,2
4,6,8,46,5,Home,Away,First,Second,Aug,6
...,...,...,...,...,...,...,...,...,...,...
2503,5,15,89,3,Neutral,Neutral,Second,First,Jun,5
2504,1,14,138,0,Home,Away,First,Second,Jan,1
2505,14,5,135,15,Home,Away,Second,First,Apr,14
2506,15,14,64,16,Home,Away,First,Second,Nov,14


In [4]:
test=pd.read_csv(path1)
test

,Team1,Team2,Stadium,HostCountry,Team1_Venue,Team2_Venue,Team1_Innings,Team2_Innings,MonthOfMatch
0,2,4,34,1,Home,Away,First,Second,Oct
1,14,1,19,15,Home,Away,First,Second,Mar
2,9,10,130,14,Neutral,Neutral,Second,First,Dec
3,9,10,8,9,Home,Away,First,Second,Dec
4,5,15,130,14,Neutral,Neutral,First,Second,Oct
...,...,...,...,...,...,...,...,...,...
1070,15,5,64,16,Home,Away,First,Second,Jun
1071,1,12,95,0,Home,Away,First,Second,Feb
1072,5,10,43,1,Neutral,Neutral,First,Second,Mar
1073,10,13,111,0,Neutral,Neutral,Second,First,Mar


In [668]:
train['team_month_stadium']=train.groupby(['Team1','Team2','Stadium'])['MonthOfMatch'].transform('nunique')
test['team_month_stadium']=test.groupby(['Team1','Team2','Stadium'])['MonthOfMatch'].transform('nunique')


In [669]:
train['Team_Hostcountry']=train.groupby(['Team1','Team2'])['HostCountry'].transform('nunique')
test['Team_Hostcountry']=test.groupby(['Team1','Team2'])['HostCountry'].transform('nunique')


In [670]:
train['Home_neutral']=train.groupby('Team1')['Team1_Venue'].transform('nunique')
test['Home_neutral']=test.groupby('Team1')['Team1_Venue'].transform('nunique')


In [671]:
train['Away_neutral']=train.groupby('Team2')['Team2_Venue'].transform('nunique')
test['Away_neutral']=test.groupby('Team2')['Team2_Venue'].transform('nunique')

In [672]:
train['team1_first_bat']=train['Team1_Innings'].str.contains('First')
test['team1_first_bat']=test['Team1_Innings'].str.contains('First')


In [673]:
train['team1_Second_bat']=train['Team1_Innings'].str.contains('Second')
test['team1_Second_bat']=test['Team1_Innings'].str.contains('Second')

In [674]:
train['team2_first_bat']=train['Team2_Innings'].str.contains('First')
train['team2_Second_bat']=train['Team2_Innings'].str.contains('Second')

test['team2_first_bat']=test['Team2_Innings'].str.contains('First')
test['team2_Second_bat']=test['Team2_Innings'].str.contains('Second')

In [675]:
ord_bat={'True':1,'False':0 }
train['team1_first_bat']=train['team1_first_bat'].astype(str)
train['team1_first_bat']=train['team1_first_bat'].map(ord_bat)


test['team1_first_bat']=test['team1_first_bat'].astype(str)
test['team1_first_bat']=test['team1_first_bat'].map(ord_bat)

In [676]:
train['team2_first_bat']=train['team2_first_bat'].astype(str)
train['team2_first_bat']=train['team2_first_bat'].map(ord_bat)


test['team2_first_bat']=test['team2_first_bat'].astype(str)
test['team2_first_bat']=test['team2_first_bat'].map(ord_bat)

In [677]:
train['team2_Second_bat']=train['team2_Second_bat'].astype(str)
train['team2_Second_bat']=train['team2_Second_bat'].map(ord_bat)


test['team2_Second_bat']=test['team2_Second_bat'].astype(str)
test['team2_Second_bat']=test['team2_Second_bat'].map(ord_bat)

In [678]:
train['team1_Second_bat']=train['team1_Second_bat'].astype(str)
train['team1_Second_bat']=train['team1_Second_bat'].map(ord_bat)


test['team1_Second_bat']=test['team1_Second_bat'].astype(str)
test['team1_Second_bat']=test['team1_Second_bat'].map(ord_bat)

In [679]:
train['team1_first_bat']=train.groupby(['Team1','Team2'])['team1_first_bat'].transform('sum')
test['team1_first_bat']=test.groupby(['Team1','Team2'])['team1_first_bat'].transform('sum')

In [680]:
train['team1_Second_bat']=train.groupby(['Team1','Team2'])['team1_Second_bat'].transform('sum')
test['team1_Second_bat']=test.groupby(['Team1','Team2'])['team1_Second_bat'].transform('sum')

In [681]:
train['team2_first_bat']=train.groupby(['Team2','Team1'])['team2_first_bat'].transform('sum')
test['team2_first_bat']=test.groupby(['Team2','Team1'])['team2_first_bat'].transform('sum')

In [682]:
train['team2_Second_bat']=train.groupby(['Team2','Team1'])['team2_Second_bat'].transform('sum')
test['team2_Second_bat']=test.groupby(['Team2','Team1'])['team2_Second_bat'].transform('sum')

In [683]:
train.drop(['Team1_Venue','Team2_Venue','Team2_Innings','Team1_Innings'],1,inplace=True)
test.drop(['Team1_Venue','Team2_Venue','Team2_Innings','Team1_Innings'],1,inplace=True)

In [684]:
train

,Team1,Team2,Stadium,HostCountry,MonthOfMatch,MatchWinner,team_month_stadium,Team_Hostcountry,Home_neutral,Away_neutral,team1_first_bat,team1_Second_bat,team2_first_bat,team2_Second_bat
0,5,4,37,4,Dec,4,3,1,2,2,12,15,15,12
1,1,14,84,7,Sep,1,1,5,2,2,39,26,26,39
2,9,15,47,9,Feb,9,1,4,2,2,7,6,6,7
3,7,2,102,6,Aug,2,2,1,2,2,4,1,1,4
4,6,8,46,5,Aug,6,2,3,2,2,2,3,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2503,5,15,89,3,Jun,5,2,7,2,2,13,16,16,13
2504,1,14,138,0,Jan,1,4,5,2,2,39,26,26,39
2505,14,5,135,15,Apr,14,1,1,2,3,12,14,14,12
2506,15,14,64,16,Nov,14,3,1,1,2,5,5,5,5


In [685]:
test

,Team1,Team2,Stadium,HostCountry,MonthOfMatch,team_month_stadium,Team_Hostcountry,Home_neutral,Away_neutral,team1_first_bat,team1_Second_bat,team2_first_bat,team2_Second_bat
0,2,4,34,1,Oct,2,1,2,2,5,1,1,5
1,14,1,19,15,Mar,1,1,2,1,3,11,11,3
2,9,10,130,14,Dec,3,4,2,2,13,14,14,13
3,9,10,8,9,Dec,4,4,2,2,13,14,14,13
4,5,15,130,14,Oct,1,4,2,2,5,2,2,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1070,15,5,64,16,Jun,3,1,1,2,7,3,3,7
1071,1,12,95,0,Feb,3,3,2,2,8,2,2,8
1072,5,10,43,1,Mar,3,6,2,2,11,16,16,11
1073,10,13,111,0,Mar,1,7,2,2,11,22,22,11


In [686]:
train.drop(['team2_first_bat', 'team2_Second_bat','team1_Second_bat', 'team1_first_bat','team_month_stadium','Stadium','team_month_stadium','HostCountry','Team_Hostcountry','MonthOfMatch'],1,inplace=True)

test.drop(['team2_first_bat', 'team2_Second_bat','team1_Second_bat', 'team1_first_bat','team_month_stadium','Stadium','team_month_stadium','HostCountry','Team_Hostcountry','MonthOfMatch'],1,inplace=True)


In [687]:
X=train.drop('MatchWinner',1)
y=train['MatchWinner']
Xt=test

In [697]:
X.columns

Index(['Team1', 'Team2', 'Home_neutral', 'Away_neutral'], dtype='object')

In [698]:
def frequency_encoding(column, df, df_test=None):
    frequencies = df[column].value_counts().reset_index()
    df_values = df[[column]].merge(frequencies, how='left', 
                                   left_on=column, right_on='index').iloc[:,-1].values
    if df_test is not None:
        df_test_values = df_test[[column]].merge(frequencies, how='left', 
                                                 left_on=column, right_on='index').fillna(1).iloc[:,-1].values
    else:
        df_test_values = None
    return df_values, df_test_values

for column in ['Team1', 'Team2', 'Home_neutral', 'Away_neutral']:
    train_values, test_values = frequency_encoding(column, X, Xt)
    X[column+'_counts'] = train_values
    Xt[column+'_counts'] = test_values

In [701]:
X.drop(['Team1','Team2','Home_neutral','Away_neutral'],1,inplace=True)
Xt.drop(['Team1','Team2','Home_neutral','Away_neutral'],1,inplace=True)


In [702]:
new_train=pd.concat([X,y],1)
new_test=Xt

In [703]:
from sklearn.metrics import log_loss
from imblearn.over_sampling import SMOTE
X=new_train.drop('MatchWinner',1)
y=new_train['MatchWinner']



xgb=XGBClassifier(random_state=9)

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=9)

xgb.fit(X_train,y_train)
y_pred=xgb.predict_proba(X_test)
LOG_LOSS=log_loss(y_test,xgb.predict_proba(X_test))
#print(model)
print('LOG LOSS:',LOG_LOSS)
cv=cross_val_score(xgb,X,y,scoring='neg_log_loss',cv=5)
print('mean score:',cv.mean())
print('std:',cv.std())
print('------------------')

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=9,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

LOG LOSS: 0.6528104427870447
mean score: -0.6658647107995134
std: 0.03291136420765711
------------------


In [704]:
feat_imp = pd.DataFrame({'columns':X.columns,'feature_importance':xgb.feature_importances_})
feat_imp

,columns,feature_importance
0,Team1_counts,0.26
1,Team2_counts,0.20
2,Home_neutral_counts,0.18
3,Away_neutral_counts,0.36


In [705]:
X.columns



Index(['Team1_counts', 'Team2_counts', 'Home_neutral_counts',
       'Away_neutral_counts'],
      dtype='object')

In [706]:
y_pred_xgb=xgb.predict_proba(test)

In [707]:
final=y_pred_xgb
final=pd.DataFrame(final)
final

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.00,0.00,0.35,0.00,0.63,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0.00,0.35,0.00,0.00,0.00,0.38,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.26,0.00
2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.51,0.48,0.00,0.00,0.00,0.00,0.00
3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.51,0.48,0.00,0.00,0.00,0.00,0.00
4,0.00,0.00,0.00,0.00,0.01,0.75,0.00,0.00,0.00,0.01,0.00,0.00,0.01,0.00,0.00,0.21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1070,0.01,0.01,0.05,0.00,0.01,0.01,0.01,0.01,0.00,0.02,0.02,0.00,0.01,0.01,0.02,0.81
1071,0.00,0.57,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.41,0.00,0.00,0.00
1072,0.00,0.00,0.00,0.00,0.00,0.42,0.00,0.00,0.00,0.01,0.56,0.00,0.00,0.00,0.00,0.00
1073,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.02,0.00,0.00,0.94,0.01,0.00


In [708]:
final.to_excel('y_pred_xgb_1.xlsx')